**This Notebook aims to categorize (A-F cat.) the pannel based on its features using a convolutionnal neural network**<br>
The model can then be used conviniently without having to re-fit it

In [11]:
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
import sklearn
from keras.utils import to_categorical
import os
import sklearn.cluster
from sklearn.model_selection import train_test_split
import itertools
import statistics
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import scipy
from scipy import signal

In [2]:
def make_labels(directory, data=[], y_hat=[], label=0):
    for root, dirs, files in os.walk(directory):
        for file in files:
            img = matplotlib.image.imread(directory+file)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = np.reshape(img, (224, 224, 1))
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)  # Convert to RGB
            img = cv2.resize(img, (224, 224))

            data.append(img)
        y_hat = [label] * len(data)
    return np.array(data), np.array(y_hat)

In [3]:
parent_folder = 'Training/augmentation1/'

a, y_a = [], []
a, y_a = make_labels(parent_folder + '/A/', data=a, y_hat=y_a, label=0)
print("a loaded")

b, y_b = [], []
b, y_b = make_labels(parent_folder + '/B/', data=b, y_hat=y_b, label=1)
print("b loaded")

c, y_c = [], []
c, y_c = make_labels(parent_folder + '/C/', data=c, y_hat=y_c, label=2)
print("c loaded")

d, y_d = [], []
d, y_d = make_labels(parent_folder + '/D/', data=d, y_hat=y_d, label=3)
print("d loaded")

e, y_e = [], []
e, y_e = make_labels(parent_folder + '/E/', data=e, y_hat=y_e, label=4)
print("e loaded")

f, y_f = [], []
f, y_f = make_labels(parent_folder + '/F/', data=f, y_hat=y_f, label=5)
print("f loaded")

a loaded
b loaded
c loaded
d loaded
e loaded
f loaded


In [4]:
print('a:', a.shape)
print('b:', b.shape)
print('c:', c.shape)
print('d:', d.shape)
print('e:', e.shape)
print('f:', f.shape)

a: (192, 224, 224, 3)
b: (192, 224, 224, 3)
c: (192, 224, 224, 3)
d: (192, 224, 224, 3)
e: (192, 224, 224, 3)
f: (192, 224, 224, 3)


**Data normalization**

In [5]:
X = np.vstack((a, b, c, d, e, f))
a = b = c = d = e = f = None
y = np.hstack((y_a, y_b, y_c, y_d, y_e, y_f)).reshape(-1, 1)
y_cat = to_categorical(y)

In [12]:
print(X.shape)
print(y_cat.shape)

(1152, 224, 224, 3)
(1152, 6)


**Create & compile the model**

In [9]:
model = Sequential()

# Add a Convolutional Layer with 32 filters, a kernel size of (3, 3), and input shape (224, 224, 3)
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))

# Add a MaxPooling Layer with pool size (2, 2)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another Convolutional Layer with 64 filters and a kernel size of (3, 3)
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another MaxPooling Layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output to feed into a Dense layer
model.add(Flatten())

# Add a Dense layer with 128 neurons
model.add(Dense(128, activation='relu'))

# Add the output layer with 6 neurons (assuming 6 categories) and softmax activation
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()

2023-12-14 18:07:00.768346: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2023-12-14 18:07:00.838344: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2023-12-14 18:07:00.897076: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Normalize pixel values to be between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

In [15]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

2023-12-14 18:09:57.957035: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 554545152 exceeds 10% of free system memory.


Epoch 1/10


2023-12-14 18:10:00.675891: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.


29/29 [==============================] - 65s 2s/step - loss: 1.7939 - accuracy: 0.1596 - val_loss: 1.7924 - val_accuracy: 0.1126
Epoch 2/10
13/29 [============>.................] - ETA: 42s - loss: 1.7914 - accuracy: 0.1947

: 